In [1]:
from typing import Literal
from tqdm import tqdm
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOllama
from langchain.agents import load_tools, initialize_agent
from langchain_community.utilities import SerpAPIWrapper

from langchain_experimental.plan_and_execute import (
    load_chat_planner, load_agent_executor, PlanAndExecute
)
from langchain.tools import Tool
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer


# if use GPT
# from langchain_openai import ChatOpenAI

import pandas as pd
import warnings

In [2]:
# load CSV
csv_name = "train_data_smi_unique_target_name"
csv_dir = "./target_csv/"
df = pd.read_csv(csv_dir + csv_name +".csv")

In [3]:
# 경고 무시 설정
warnings.filterwarnings("ignore", category=FutureWarning)

# 토크나이저 설정
# # clean_up_tokenization_spaces=True, TRUE => 공백 포함, FALSE => 공백 제거, default = TRUE, 글 => TRUE, 코드나 포맷 형식 => FALSE  
# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# 임베딩 모델 로드
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [4]:
ReasoningStrategies = Literal["zero-shot-react", "plan-and-solve"]

def load_agent(
    tool_names: list[str],
    strategy: ReasoningStrategies = "zero-shot-react"
) -> LLMChain:
    llm = ChatOllama(model="llama3.1:70b", temperature=0, streaming=True)
    tools = load_tools(
        tool_names=tool_names,
        llm=llm
    )
    
    if strategy == "plan-and-solve":
        planner = load_chat_planner(llm)
        executor = load_agent_executor(llm, tools, verbose=True)
        return PlanAndExecute(planner=planner, executor=executor, verbose=True)

    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

In [5]:
# tools_llm = ChatOllama(model="llama3.1:70b", temperature=0, streaming=True)

In [6]:
# 다른 도구들과 함께 PubMed Tool 로드
tools = ["wikipedia", "serpapi", "pubmed"]

In [7]:
# 에이전트 초기화
agent_chain = load_agent(tools, strategy="plan-and-solve")

In [8]:
# Description 컬럼을 생성하여 각 protein_name에 대한 설명을 추가 및 저장
if "description" not in df.columns:
    df["description"] = ""

In [9]:
save_threshold = 10  # 열 개마다 저장
save_count = 0

In [10]:
for idx, protein_name in tqdm(enumerate(df["Target Name"]), desc="Fetching descriptions", total=len(df)):
    if not df.at[idx, "description"]:  # 이미 설명이 채워진 경우 건너뜀
        query = f"Please provide a brief description of {protein_name}."
        try:
            description = agent_chain.invoke(query, handle_parsing_errors=True)

            if isinstance(description, dict) and 'output' in description:
                description = description['output']
            else:
                description = f"Error: Unexpected format - {description}"
                
        except ValueError as e:
            description = f"Error: {str(e)}"
        
        df.at[idx, "description"] = description
        save_count += 1

        # 열 개의 설명이 추가되면 저장
        if save_count >= save_threshold:
            df.to_csv(csv_name + " cnt," + str(save_count) + ".csv", index=False)
            save_count = 0  # 저장 후 카운트 초기화

Fetching descriptions:   0%|                                                        | 0/61 [00:00<?, ?it/s]



> Entering new PlanAndExecute chain...
steps=[Step(value='Research and gather information on Tyrosine-protein kinase receptor RET from credible sources such as scientific journals, databases, or reputable websites.'), Step(value='Identify the key characteristics, functions, and roles of RET in cellular processes and human health.'), Step(value="Organize the gathered information into a concise and accurate description.\n\nGiven the above steps taken, please respond to the user's original question:\n\nTyrosine-protein kinase receptor RET (RET) is a proto-oncogene that encodes a receptor tyrosine kinase for glial cell line-derived neurotrophic factor (GDNF) family of extracellular signaling molecules. It plays a crucial role in the development and maintenance of the nervous system, particularly in the formation and survival of neurons. RET is also involved in the regulation of cell growth, differentiation, and migration.\n\nRET mutations have been associated with multiple endocrine neop

Fetching descriptions:   2%|▊                                             | 1/61 [01:10<1:10:34, 70.57s/it]



> Entering new PlanAndExecute chain...
steps=[Step(value='Research and gather information on Macrophage-stimulating protein receptor (MST1R) from credible sources.'), Step(value='Organize the gathered information into a concise and clear format.'), Step(value="Given the above steps taken, please respond to the user's original question.\n\n")]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "wikipedia",
  "action_input": "Macrophage-stimulating protein receptor"
}
```


Observation: Page: Granulocyte-macrophage colony-stimulating factor receptor
Summary: The granulocyte-macrophage colony-stimulating factor receptor, also known as CD116 (Cluster of Differentiation 116), is a receptor for granulocyte-macrophage colony-stimulating factor, which stimulates the production of white blood cells. In contrast to M-CSF and G-CSF which are lineage specific, GM-CSF and its receptor play a role in earlier stages of development. The receptor is primarily located on neutrophils, eos

Fetching descriptions:   3%|█▌                                              | 2/61 [01:51<52:04, 52.97s/it]

...

Tyrosine-protein kinase BRK (Breast Tumor Kinase) is a non-receptor tyrosine kinase that plays a crucial role in cell signaling pathways, particularly in the regulation of cellular processes such as proliferation, differentiation, and survival. It is primarily expressed in epithelial cells and has been implicated in the development and progression of various cancers, including breast cancer.

> Finished chain.
*****

Step: Organize the gathered information into a concise and accurate description.

Given the above steps taken, please respond to the user's original question:

Tyrosine-protein kinase BRK (Breast Tumor Kinase) is a non-receptor tyrosine kinase that plays a crucial role in cell signaling pathways, particularly in the regulation of cellular processes such as proliferation, differentiation, and survival. It is primarily expressed in epithelial cells and has been implicated in the development and progression of various cancers, including breast cancer.



Response: Tyrosine-pr

Fetching descriptions:  98%|██████████████████████████████████████████████▏| 60/61 [45:49<00:43, 43.79s/it]

Serine/threonine-protein kinase AKT3 is a member of the AKT family of serine/threonine kinases that plays a crucial role in cell survival, growth, and metabolism. It phosphorylates various substrates involved in these processes and is implicated in the PI3K/AKT signaling pathway. AKT3 is expressed in various tissues, including brain, heart, and skeletal muscle, and localizes to the cytoplasm and nucleus.

> Finished chain.
*****

Step: Summarize key points about Serine/threonine-protein kinase AKT3.

Given the above steps taken, please respond to the user's original question:

Serine/threonine-protein kinase AKT3 is a member of the AKT family of serine/threonine kinases that plays a crucial role in cell survival, growth, and metabolism. It phosphorylates various substrates involved in these processes and is implicated in the PI3K/AKT signaling pathway. AKT3 is expressed in various tissues, including brain, heart, and skeletal muscle, and localizes to the cytoplasm and nucleus.



Respo

Fetching descriptions: 100%|███████████████████████████████████████████████| 61/61 [46:18<00:00, 45.55s/it]

The PI3-kinase p110-gamma subunit is an isoform of the phosphoinositide 3-kinase (PI3K) enzyme that plays a specific role in cell signaling pathways, particularly in regulating inflammation and immune responses. It forms a heterodimer with regulatory subunits to form the active PI3K enzyme complex.

> Finished chain.
*****

Step: Gather information on its structure and interactions: The p110-gamma subunit forms a heterodimer with a regulatory subunit, typically p101 or p84/87, to form the active PI3K enzyme complex.

Given the above steps taken, please respond to the user's original question:

The PI3-kinase p110-gamma subunit is an isoform of the phosphoinositide 3-kinase (PI3K) enzyme that plays a specific role in cell signaling pathways, particularly in regulating inflammation and immune responses. It forms a heterodimer with regulatory subunits to form the active PI3K enzyme complex.



Response: The PI3-kinase p110-gamma subunit is an isoform of the phosphoinositide 3-kinase (PI3K

In [11]:
# 모든 작업이 끝난 후 마지막으로 저장 (남아있는 데이터 저장)
if save_count > 0:
    df.to_csv(csv_dir+csv_name + "_descriptions.csv", index=False)

In [15]:
df = df.drop(index=0)

In [16]:
df

,Target Name,description
1,Macrophage-stimulating protein receptor,Here is a concise and clear format of the gath...
2,Cyclin-dependent kinase 7,Cyclin-dependent kinase 7 (CDK7) is an enzyme ...
3,Serine-protein kinase ATM,"Unfortunately, I don't have any information ab..."
4,Plasma kallikrein,Plasma kallikrein is a serine protease that pl...
5,PI3-kinase p110-alpha subunit,The PI3-kinase p110-alpha subunit (PIK3CA) is ...
6,Signal transducer and activator of transcripti...,STAT3 is a protein that plays a crucial role i...
7,Rho-associated protein kinase 2,ROCK2 is an enzyme that plays a crucial role i...
8,Tyrosine-protein kinase JAK2,Tyrosine-protein kinase JAK2 is a non-receptor...
9,Neurotrophic tyrosine kinase receptor type 2,Tyrosine-protein kinase JAK2 is a non-receptor...
10,Serine/threonine-protein kinase B-raf,"Serine/threonine-protein kinase B-raf, also kn..."


In [12]:
# # JSon, output만 분리
# df['description'] = df['description'].apply(lambda x: x['output'])

In [18]:
# 임베딩을 Target Name에 대응시키는 딕셔너리 생성
df['embedding'] = df['description'].apply(
    lambda text: None if text is None else embedding_model.encode(text)
)

In [19]:
df.to_csv(csv_dir + csv_name + "_descriptions.csv", index=False)

In [ ]:
# # 예시 도구 이름 리스트
# tool_names = ["wikipedia"]

# # 에이전트 로드 (제로샷 모델 사용)
# agent_chain = load_agent(tool_names, strategy="zero-shot-react")

# # 에이전트 실행
# result = agent_chain.run("Explain about T1-Faker")

# print("Result:\n", result)

In [ ]:
# from langchain_experimental.plan_and_execute import PlanAndExecute
# from langchain.llms import OpenAI
# from langchain.tools import Tool
# from langchain_community.utilities import SerpAPIWrapper

# # LLM 초기화
# llm = OpenAI(model="gpt-4")

# # PubMed 검색 함수 정의
# def search_pubmed(query: str) -> str:
#     serpapi = SerpAPIWrapper()
#     results = serpapi.run(query)
#     return "\n".join(results)

# # PubMed 도구 생성
# pubmed_tool = Tool(
#     name="pubmed_search",
#     func=search_pubmed,
#     description="Search PubMed for scientific articles related to the query."
# )

# # Plan-and-Execute 에이전트 초기화
# agent = PlanAndExecute(
#     planner=llm,
#     executor=llm,
#     tools=[pubmed_tool],
#     verbose=True
# )

# # 에이전트를 사용한 쿼리 실행
# query = "Tyrosine-protein kinase receptor RET"
# response = agent.run(query)

# print(response)
